In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [80]:
# Some more imports
##
import warnings
warnings.filterwarnings("ignore")

In [34]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
sample_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
sample_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [35]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [36]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [37]:
train['text'].sample(30)

2987    What This Man Did To Save A Drowning Squirrel ...
4672    Punjab government flood relief platform: http:...
3491    #cum explosion!\n\n@begforcum \n@allday_cumsho...
3513    #Eyewitness media is actively embraced by #UK ...
3688                     Fatality https://t.co/GF5qjGoyCi
5417    I feel like I should be panicking more as Idk ...
3056    #USGS M 1.4 - 4km E of Interlaken California: ...
1551    Downtown Emergency Service Center is hiring! #...
6196    @LifeAintFairKid if I did I'd smoke you up brooo!
6940    If you have trouble getting motivated remember...
4872    Not only are you a mass murderer but at a movi...
3274    [Question] Is anybody else having this problem...
2147    Weyburn Police Warn Public after Fentanyl Deat...
616     A Tale of Two Pox - Body Horrors http://t.co/W...
4997    13 reasons why we love women in the military  ...
3745       My asshole is on fire  https://t.co/Y3FO0gHg8t
397     Mourning notices for stabbing arson victims st...
7407    Prince

## Some EDA

Missing data

In [38]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [39]:
train['keyword'].value_counts()[:10]

fatalities     45
deluge         42
armageddon     42
damage         41
sinking        41
harm           41
body%20bags    41
evacuate       40
twister        40
fear           40
Name: keyword, dtype: int64

In [40]:
# A list to randomly fill the missing keywords
##
kwords = ['fatalities', 'deluge', 'armageddon', 'damage', 'sinking', 'harm', 'body%20bags', 
          'evacuate', 'twister', 'fear']

In [74]:
type(train['keyword'][0]) == float

True

In [75]:
# Impute rows with missing keyword
##
for i, kword in enumerate(train['keyword']):
    if type(kword) == float: # Note that type of nan is float
        train['keyword'][i] = np.random.choice(kwords)
    else:
        pass

# Checking to see that the rows have been filled
train['keyword'].isnull().values.any()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


False

In [78]:
# Preview missing  keywords in test data
##
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [79]:
# Same treatment to the test data
##
for i, kword in enumerate(test['keyword']):
    if type(kword) == float: # Note that type of nan is float
        test['keyword'][i] = np.random.choice(kwords)
    else:
        pass

# Checking to see that the rows have been filled
test['keyword'].isnull().values.any()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


False

## Text Processing

**Text processing require:**  
- remove punctuation  
- remove numbers  
- to lowercase  
- remove stopwords  
- lemmatize  
- remove non-english words  

**NOTE** : The order of the steps matter. The text processing function should execute steps in this outlined order.

In [82]:
# A function to process text as outlined above

import string
import nltk
from nltk.corpus import stopwords

words = set(nltk.corpus.words.words())

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def text_process(text):
    cln_text = [c for c in text if c not in string.punctuation] # punctuation removal
    cln_text = ''.join(cln_text)
    cln_text = [c for c in cln_text if not c.isdigit()] # number removal
    cln_text = ''.join(cln_text)
    cln_text = cln_text.lower() # lowercase
    cln_text = [word for word in cln_text.split() if word not in stopwords.words('english')] # stopwords removal
    cln_text = ' '.join(cln_text)
    cln_text = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(cln_text)] # lemmatization
    cln_text = ' '.join(cln_text)
    cln_text = [word for word in nltk.wordpunct_tokenize(cln_text) if word in words or not word.isalpha()] # removal of non-english words
    return cln_text

In [88]:
# Example of how the function works on a single text
##
text_process(train['text'][0])

['deed', 'reason', 'earthquake', 'may', 'forgive', 'u']

In [89]:
# BOW model
##
from sklearn.feature_extraction.text import CountVectorizer

bow_transformer = CountVectorizer(analyzer=text_process)
bow_transformer.fit(train['text'])

CountVectorizer(analyzer=<function text_process at 0x7f25fb8ea1e0>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [90]:
# Number of BOW vocab words
##
len(bow_transformer.vocabulary_)

6702

In [91]:
# Transforming a single text
##
print(bow_transformer.transform([train['text'][1230]]))

  (0, 325)	1
  (0, 1048)	1
  (0, 2099)	1
  (0, 2248)	1
  (0, 2698)	1
  (0, 3017)	1


In [92]:
# Getting feature names
##
bow_transformer.get_feature_names()[1048]

'cladding'

In [93]:
# Transforming the whole BOW corpus into a sparse matrix
##
bow_text = bow_transformer.transform(train['text'])

In [100]:
# Shape of the matrix
##
bow_text.shape

(7613, 6702)

In [101]:
#tfidf weighting
##
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(bow_text)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [102]:
# Transforming a single BOW
##
print(tfidf_transformer.transform(bow_transformer.transform([train['text'][1230]])))

  (0, 3017)	0.4700906051478796
  (0, 2698)	0.34633250839624347
  (0, 2248)	0.22818049172662036
  (0, 2099)	0.43932738517349695
  (0, 1048)	0.4700906051478796
  (0, 325)	0.43932738517349695


In [103]:
# Getting idf of a word
##
tfidf_transformer.idf_[bow_transformer.vocabulary_['cladding']]

8.551449575822552

In [104]:
# Transforming the whole sparse matrix
##
tfidf_text = tfidf_transformer.transform(bow_text)

In [105]:
tfidf_text.shape

(7613, 6702)

## keyword as a feature

In [109]:
# building a sparse matrix of keywords
##
kword_cv = CountVectorizer()
kword_cv.fit(train['keyword'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [112]:
# keywords BOW
##
kword_bow = kword_cv.transform(train['keyword'])

In [113]:
# tfidf keyword weights
##
tfidf_transformer.fit(kword_bow)

kword_tfidf = tfidf_transformer.transform(kword_bow)

Combining the two sparse matrices (text and keyword)

In [115]:
from scipy.sparse import hstack

features = hstack([tfidf_text, kword_tfidf])

**ML Algorithm Selection**

In [116]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [117]:
# So, a classification algorithm eg. Logistic regression, naive bayes

In [118]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [119]:
# train test split
##
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, train['target'], test_size=0.3, random_state=101)

In [120]:
# imports for model evaluation
from sklearn.metrics import confusion_matrix, classification_report

In [121]:
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

In [122]:
# Model evaluations
##

print("---------------------------------")
print("Logistic Regression")
print("---------------------------------")
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))
print("---------------------------------")
print("Naive Bayes MultinomialNB")
print("---------------------------------")
print(confusion_matrix(y_test, nb_pred))
print(classification_report(y_test, nb_pred))
print("---------------------------------")
print("RandomForestClassifier")
print("---------------------------------")
print(confusion_matrix(y_test, rfc_pred))
print(classification_report(y_test, rfc_pred))

---------------------------------
Logistic Regression
---------------------------------
[[1160  170]
 [ 287  667]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.84      1330
           1       0.80      0.70      0.74       954

    accuracy                           0.80      2284
   macro avg       0.80      0.79      0.79      2284
weighted avg       0.80      0.80      0.80      2284

---------------------------------
Naive Bayes MultinomialNB
---------------------------------
[[1111  219]
 [ 258  696]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.82      1330
           1       0.76      0.73      0.74       954

    accuracy                           0.79      2284
   macro avg       0.79      0.78      0.78      2284
weighted avg       0.79      0.79      0.79      2284

---------------------------------
RandomForestClassifier
---------------------------------
[[1152  178]
 [ 320

In [61]:
# Logistic regression did quite well than the other two

**Building a data pipeline to predict on the test data.**

In [123]:
# Combining the features in one helper function
##
from sklearn.compose import ColumnTransformer

coltrans = ColumnTransformer([
    ("text bow", CountVectorizer(analyzer=text_process), 'text'),
    ("kword bow", CountVectorizer(), 'keyword')],
    remainder = 'drop'
)

In [124]:
coltrans.fit(train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('text bow',
                                 CountVectorizer(analyzer=<function text_process at 0x7f25fb8ea1e0>,
                                                 binary=False,
                                                 decode_error='strict',
                                                 dtype=<class 'numpy.int64'>,
                                                 encoding='utf-8',
                                                 input='content',
                                                 lowercase=True, max_df=1.0,
                                                 max_features=None, min_df=1,
                                                 ngram_range=(1, 1),
                                                 pr...
                                ('kword bow',
                                 CountVectorizer(analyzer='word', bi

In [130]:
# tfidf off of the BOW produced by transforming the features as below
##
tfidf_train = coltrans.transform(train)

In [131]:
print(features.shape)
print(tfidf_train.shape)

(7613, 6941)
(7613, 6941)


In [149]:
# PIPELINE

from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("coltrans", ColumnTransformer([
        ("text bow", CountVectorizer(analyzer=text_process), 'text'),
        ("kword bow", CountVectorizer(), 'keyword')],
        remainder = 'drop')),
    ("tfidf", TfidfTransformer()),
    ("clf", LogisticRegression())
])

# Designing pipe1 and pipe2 for naive bayes and random forest 
# They are similar to the pipeline above apart from the classifier

pipe1 = Pipeline([
    ("coltrans", ColumnTransformer([
        ("text bow", CountVectorizer(analyzer=text_process), 'text'),
        ("kword bow", CountVectorizer(), 'keyword')],
        remainder = 'drop')),
    ("tfidf", TfidfTransformer()),
    ("clf", MultinomialNB())
])

pipe2 = Pipeline([
    ("coltrans", ColumnTransformer([
        ("text bow", CountVectorizer(analyzer=text_process), 'text'),
        ("kword bow", CountVectorizer(), 'keyword')],
        remainder = 'drop')),
    ("tfidf", TfidfTransformer()),
    ("clf", RandomForestClassifier())
])


In [135]:
text_train, text_test, label_train, label_test = train_test_split(train[['text', 'keyword']], train['target'],
                                                                  test_size=0.3, random_state=101) 

In [137]:
pipe.fit(text_train, label_train)

Pipeline(memory=None,
         steps=[('coltrans',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text bow',
                                                  CountVectorizer(analyzer=<function text_process at 0x7f25fb8ea1e0>,
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df

In [142]:
pipe_pred = pipe.predict(test)

In [143]:
pipe_pred.shape

(3263,)

In [144]:
test.shape

(3263, 4)

In [145]:
submission3 = pd.DataFrame({
    "id": test['id'],
    "target": pipe_pred
})

In [150]:
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [147]:
submission3

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [148]:
submission3.to_csv('submission3.csv', index=False)

In [151]:
# More submissions from pipe1 and pipe2
##
# pipe1
pipe1.fit(text_train, label_train)

# Prediction on test data
pipe1_pred = pipe1.predict(test)

# pipe2
pipe2.fit(text_train, label_train)

pipe2_pred = pipe2.predict(test)

In [152]:
submission4 = pd.DataFrame({
    "id": test['id'],
    "target": pipe1_pred
})

submission5 = pd.DataFrame({
    "id": test['id'],
    "target": pipe2_pred
})

In [ ]:
submission4.to_csv("submission4.csv", index=False)
submission5.to_csv("submission5.csv", index=False)